```Prolog
% Fact
parent('범근', '두리').
parent('두리', '철수').
parent('순자', '철수').
parent('철수', '돌이').
parent('영희', '돌이').
parent('철수', '민순').
parent('영희', '민순').
% Rule
ancestor(A, X) :- parent(A, X).
ancestor(A, X) :- parent(P, X), ancestor(A, P).
grandparent(G, X) :- parent(G, P), parent(P, X).
sibling(X, Y) :- parent(P, X), parent(P, Y), X\=Y.
```

위와 같은 Prolog 프로그램에서 재귀적으로 정의된 `ancestor` 관계를 Haskell로 옮겨보자.

참고로 관계를 하스켈에서 순서쌍의 리스트로 정의하는 방법에 대해서는 같은 저장소에 들어있는 Relation 노트북을 참고하면 좋다.

In [10]:
import Data.List

parentRel = [("범근", "두리"),
             ("두리", "철수"),
             ("순자", "철수"),
             ("철수", "돌이"),
             ("영희", "돌이"),
             ("철수", "민순"),
             ("영희", "민순")]

printRel = mapM_ (\(x,y) -> do{putStr x; putStr ", "; putStrLn y})

rel1 = parentRel
rel2 = [(z,x) | (y,x) <- parentRel, (z,y')<-rel1, y==y']
rel3 = [(z,x) | (y,x) <- parentRel, (z,y')<-rel2, y==y']
rel4 = [(z,x) | (y,x) <- parentRel, (z,y')<-rel3, y==y']

ancestorRelManual = rel1 `union` rel2 `union` rel3 `union` rel4

ancestorRel = transitiveClosure parentRel

In [11]:
printRel ancestorRel

범근, 두리
두리, 철수
순자, 철수
철수, 돌이
영희, 돌이
철수, 민순
영희, 민순
범근, 철수
두리, 돌이
순자, 돌이
두리, 민순
순자, 민순
범근, 돌이
범근, 민순

In [12]:
map length [rel1, rel2, rel3, rel4]

[7,5,2,0]

In [13]:
transCl rel 1 = rel
transCl rel n = [(z,x) | (y,x) <- rel, (z,y')<-transCl rel (n-1), y==y']

In [14]:
take 4 $ map (length . transCl parentRel) [1..]

[7,5,2,0]

In [15]:
foldr union [] $ takeWhile (not . null) $ map (transCl parentRel) [1..]
printRel it

[("\48276\44540","\46160\47532"),("\46160\47532","\52384\49688"),("\49692\51088","\52384\49688"),("\52384\49688","\46028\51060"),("\50689\55148","\46028\51060"),("\52384\49688","\48124\49692"),("\50689\55148","\48124\49692"),("\48276\44540","\52384\49688"),("\46160\47532","\46028\51060"),("\49692\51088","\46028\51060"),("\46160\47532","\48124\49692"),("\49692\51088","\48124\49692"),("\48276\44540","\46028\51060"),("\48276\44540","\48124\49692")]

범근, 두리
두리, 철수
순자, 철수
철수, 돌이
영희, 돌이
철수, 민순
영희, 민순
범근, 철수
두리, 돌이
순자, 돌이
두리, 민순
순자, 민순
범근, 돌이
범근, 민순

In [16]:
transitiveClosure rel = foldr union []
                      $ takeWhile (not . null)
                      $ map (transCl rel) [1..]

In [17]:
printRel $ transitiveClosure parentRel

범근, 두리
두리, 철수
순자, 철수
철수, 돌이
영희, 돌이
철수, 민순
영희, 민순
범근, 철수
두리, 돌이
순자, 돌이
두리, 민순
순자, 민순
범근, 돌이
범근, 민순

In [18]:
pRel = [("민순", "범근"),
        ("범근", "두리"),
        ("두리", "철수"),
        ("순자", "철수"),
        ("철수", "돌이"),
        ("영희", "돌이"),
        ("철수", "민순"),
        ("영희", "민순")]

printRel = mapM_ (\(x,y) -> do{putStr x; putStr ", "; putStrLn y})

r1 = pRel
r2 = [(z,x) | (y,x) <- pRel, (z,y')<-r1, y==y']
r3 = [(z,x) | (y,x) <- pRel, (z,y')<-r2, y==y']
r4 = [(z,x) | (y,x) <- pRel, (z,y')<-r3, y==y']
r5 = [(z,x) | (y,x) <- pRel, (z,y')<-r4, y==y']
r6 = [(z,x) | (y,x) <- pRel, (z,y')<-r5, y==y']

-- ancestorRelManual = rel1 `union` rel2 `union` rel3 `union` rel4

In [19]:
map length [r1, r2, r3, r4, r5, r6]

[8,8,7,7,8,8]

In [32]:
-- 싸이클을 포함한 관계의 전이 폐포 관계 만들기
-- set: 지금까지 찾은 조상 관계를 다 모아 놓은 것

transitive r = head [r1 | (r1,r2)<-zip tlist (tail tlist), r1==r2]
  where
  tlist = map (trans r r) [0..]

trans set r 0 = set
trans set r n = trans set1 r (n-1)
   where set1 = set `union` [(z,x) | (y,x) <- r, (z,y')<-set, y==y']

In [33]:
length $ trans pRel pRel 0
length $ trans pRel pRel 1
length $ trans pRel pRel 2
length $ trans pRel pRel 3
length $ trans pRel pRel 4
length $ trans pRel pRel 5

8

16

23

30

30

30

In [34]:
printRel $ transitive pRel

민순, 범근
범근, 두리
두리, 철수
순자, 철수
철수, 돌이
영희, 돌이
철수, 민순
영희, 민순
철수, 범근
영희, 범근
민순, 두리
범근, 철수
두리, 돌이
순자, 돌이
두리, 민순
순자, 민순
두리, 범근
순자, 범근
철수, 두리
영희, 두리
민순, 철수
범근, 돌이
범근, 민순
범근, 범근
두리, 두리
순자, 두리
철수, 철수
영희, 철수
민순, 돌이
민순, 민순